In [4]:
from rcv_distribution import *

In [8]:
points = get_consistency_points("Alaska_11082022_USRepresentative.csv")

/Users/es5891/Documents/GitHub/bugs-in-democracy/rcv_learning/rcv_dimensionality.py:163: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)
